In [2]:
import gymnasium as gym
from model import Model_Class

env = gym.make("InvertedPendulum-v4")
model = Model_Class(env)
# model.learn(1e5)
model.save("testrun")

Saving


In [3]:
env = gym.make("InvertedPendulum-v4", render_mode="human")
env.reset()

while True:
    finished = False
    observation, _ = env.reset()
    reward = 0
    while not finished:
        action, _ = model.get_action(observation, deterministic=True)
        observation, reward_next, terminated, truncated, _ = env.step(action)
        finished = truncated or terminated
        reward += reward_next

    print(f"epoch reward = {reward}") 

AttributeError: 'Model_Class' object has no attribute 'get_action'

: 